In [2]:
import time
import requests
import json
from pprint import pprint

In [24]:
WEATHER_URL="https://api.open-meteo.com/v1/forecast"
LATITUDE = 53.31
LONGITUDE = -6.22

# class Station:
#     def __init__(self,lat,lon):
#         self.lat = lat
#         self.lon = lon

weather=requests.get(WEATHER_URL,params={"latitude":LATITUDE,"longitude":LONGITUDE,"hourly":"temperature_2m","hourly":"precipitation_probability","hourly":"precipitation","hourly":"weathercode","hourly":"windspeed_10m","current_weather":"true","timeformat":"unixtime"})
weather_dict=json.loads(weather.text)
pprint(json.loads(weather.text))

{'current_weather': {'temperature': 11.1,
                     'time': 1677250800,
                     'weathercode': 51,
                     'winddirection': 341.0,
                     'windspeed': 19.0},
 'elevation': 15.0,
 'generationtime_ms': 0.538945198059082,
 'hourly': {'time': [1677196800,
                     1677200400,
                     1677204000,
                     1677207600,
                     1677211200,
                     1677214800,
                     1677218400,
                     1677222000,
                     1677225600,
                     1677229200,
                     1677232800,
                     1677236400,
                     1677240000,
                     1677243600,
                     1677247200,
                     1677250800,
                     1677254400,
                     1677258000,
                     1677261600,
                     1677265200,
                     1677268800,
                     1677272400,
    

In [25]:
print(weather_dict['current_weather'])

{'temperature': 11.1, 'windspeed': 19.0, 'winddirection': 341.0, 'weathercode': 51, 'time': 1677250800}


In [ ]:
print(weather_dict['hourly'])